### _Stubborn del_

#### Example 01

In [1]:
class SomeClass:
    def __del__(self):
        print('Deleted')

In [3]:
x = SomeClass()
y = x
del x # this should print 'Deleted'
del y

Deleted


#### Example 02

In [10]:
x = SomeClass()
y = x
del x
print(y) # check if y exists
del y

Deleted


#### Explanation

- `del x` doesn't directly call `x.__del__()`
- When `del x` is encountered, Python deletes the name `x` from the current scope and decrements by 1 the reference count of the object `x` referenced. `__del__()` is called only when the object’s reference count reaches zero.
- `del Name` only calls when references count reached zero.

### _The Out of Scope Variable_

In [3]:
a = 1
def some_func():
    print(a) # 1

def another_func():
    a += 1
    print(a) # UnboundLocalError

some_func()
another_func()

1


UnboundLocalError: local variable 'a' referenced before assignment

#### Explanation

- When you make an assignment to a variable in scope, it becomes local to that scope. So `a` becomes local to the scope of `another_func`, but it hasn't been initialized previously in the same scope.
- https://sebastianraschka.com/Articles/2014_python_scope_and_namespaces.html

In [6]:
# to modify the outer scope variable a use global keyword

a = 1
def another_func():
    global a
    a += 1
    print(a)

another_func()

2


### _Deleting a List Item While Iterating_

In [7]:
# https://stackoverflow.com/questions/45946228/what-happens-when-you-try-to-delete-a-list-element-while-iterating-over-it
# https://stackoverflow.com/questions/45877614/how-to-change-all-the-dictionary-keys-in-a-for-loop-with-d-items

list_1 = list(range(1,5))
list_2 = list(range(1,5))
list_3 = list(range(1,5))
list_4 = list(range(1,5))

for index, item in enumerate(list_1):
    del item

for index, item in enumerate(list_2):
    list_2.remove(item)

for index, item in enumerate(list_3[:]): # make a new copy of list
    list_3.remove(item)

for index, item in enumerate(list_4):
    list_4.pop(index)

print(list_1)
print(list_2)
print(list_3)
print(list_4)

[1, 2, 3, 4]
[2, 4]
[]
[2, 4]


In [9]:
x = list(range(10))
x.pop(5)
print(x) # 0,1,2,3,4,6,7,8,9

x.remove(6)
print(x) # 0,1,2,3,4,7,8,9

[0, 1, 2, 3, 4, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 7, 8, 9]


#### Explanation

Difference between `del` | `pop` | `remove`

- `del var_name` just removes the binding of the `var_name` from the local or global scope.
- `remove` removes the first matching value. Raise `ValueError` if the value is not found.
- `pop` removes & return at a specific index value. Raise `IndexError` if the index is not valid

Why result `[2, 4]`

- The list iteration is done index by index, and when we remove `1` from `list_2` or `list_4`, the contents of the lists are now `[2, 3, 4]`. The remaining elements are shifted down, i.e.,`2` is at index `0`, and `3` is at index `1`. 
- Since the next iteration is going to look at index `1` (which is the 3), the `2` gets skipped entirely. A similar thing will happen with every alternate element in the list sequence.

### _A Tic-tac-toe Where X Wins in the First Attempt!_

In [10]:
empty_cell = ''
row = [empty_cell] * 3 # ['', '', '']
board = [row] * 3 # [['', '', ''], [['', '', '']], [['', '', '']]]

print(board) # [['', '', ''], [['', '', '']], [['', '', '']]]
print(board[0]) # ['', '', '']
print(board[0][0]) # ''
board[0][0] = 'X'
print(board)

[['', '', ''], ['', '', ''], ['', '', '']]
['', '', '']

[['X', '', ''], ['X', '', ''], ['X', '', '']]


#### Explanation

- when `board` is initialized by multiplying the `row`, each of the elements `board[0], board[1], board[2]` is a reference to the same list referred by `row`. Because `str` is immutable...
- so when we modified `board[0][0]` we've technically modified `row[0]`. Which means we've also modified `board[1][0], board[2][0]` as well since they're pointing to the same object...
- Just like the `board` is composed of the `row`, the `row` is composed of `empty_cell`. So why didn’t `row[0]` lead to the modification of `empty_cell`, and ultimately to the entire row? It’s because `empty_cell` is an immutable variable, whereas row is a mutable variable. The behavior only applies to mutable variables.

In [18]:
# how to Avoid

empty_cell = ''
board = [[empty_cell]*3 for _ in range(3)] # [['', '', ''], [['', '', '']], [['', '', '']]]

print(board) # [['', '', ''], [['', '', '']], [['', '', '']]]
print(board[0]) # ['', '', '']
print(board[0][0]) # ''
board[0][0] = 'X'
print(board)

[['', '', ''], ['', '', ''], ['', '', '']]
['', '', '']

[['X', '', ''], ['', '', ''], ['', '', '']]


### _Lossy Zip of Iterators_

In [22]:
numbers = list(range(7))
print(numbers) # 0,1,2,3,4,5,6

first_three, remianing = numbers[0:3], numbers[3:]
print(first_three, remianing) # [0,1,2] [3,4,5,6]

numbers_iter = iter(numbers)
print(numbers_iter)
print(list(zip(numbers_iter, first_three))) # [(0,0), (1,1), (2,2)]

print(list(zip(numbers_iter, remianing))) # ?where 3 goes

[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2] [3, 4, 5, 6]
[(0, 0), (1, 1), (2, 2)]
[(4, 3), (5, 4), (6, 5)]


In [ ]:
# zip implementation

def zip(*iterables):
    sentinel = object()
    iterators = [iter(it) for it in iterables]
    while iterators:
        result = []
        for it in iterators:
            elem = next(it, sentinel)
            if elem is sentinel: return
            result.append(elem)
        yield tuple(result)

#### I didn't understand this explanation

- So, the function takes in an arbitrary number of iterable objects, adds each of their items to the result list by calling the next function on them, and stops whenever any of the iterable objects are exhausted.
- The caveat here is that when any iterable is exhausted, the existing elements in the result list are discarded. That’s what happened with 3 in the numbers_iter.

In [29]:
# How to Avoid

numbers = list(range(7))
first_three, remianing = numbers[0:3], numbers[3:]
numbers_iter = iter(numbers)

# first arg of zip should be the one with fewest element
print(list(zip(first_three, numbers_iter)))
# print(list(numbers_iter)) [3,4,5,6]
print(list(zip(remianing, numbers_iter)))

[3, 4, 5, 6]
[(0, 0), (1, 1), (2, 2)]
[(3, 3), (4, 4), (5, 5), (6, 6)]
